In [86]:
#%%
# https://selenium-python.readthedocs.io/api.html

# https://www.w3schools.com/python/python_regex.asp
    
# https://www.techbeamers.com/selenium-webdriver-waits-python/

# https://stackoverflow.com/questions/55400703/how-to-scroll-down-in-youtube-using-selenium
#
import pip
import time
import os
import pandas as pd
import datetime as dt
import requests
import json
import numpy as np
from selenium.webdriver.common.keys import Keys
from operator import itemgetter 
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from operator import itemgetter 
import sys
from IPython.display import clear_output
%matplotlib inline

def update_progress(progress, desc_string):
    bar_length = 60
    
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}% Elapsed Run Time: {2}".format( "#" * block + "-" * (bar_length - block), 
                                                                   progress * 100,
                                                                   desc_string)
    print(text)

try:
    from bs4 import BeautifulSoup as bs
except ImportError:
    pip.main(['install', 'bs4'])
    from bs4 import BeautifulSoup as bs
    
try:
    from selenium import webdriver
    from selenium.common.exceptions import TimeoutException
except ImportError:
    pip.main(['install', 'selenium'])
    from selenium import webdriver
    from selenium.common.exceptions import TimeoutException

from selenium import webdriver
chrome_dir = os.getcwd()

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

##################### ADD SHOW NAMES TO LIST HERE #########################

show_aliases = pd.read_csv('shows_list.csv')

In [177]:
# Function for getting posts information

def get_post_df(browser, posts, code_sleep_time = 0.1):

    post_df = pd.DataFrame(columns = ['title','vid_length','approx_views','approx_comments','vid_url'])
    
    failed_posts = pd.DataFrame(columns = ['title','vid_url'])
    
    start_time = time.time()

    for i, post in enumerate(posts):
        try:
            actions = ActionChains(browser)
            title = post.find_element_by_class_name('pu1E0').text
            vid_length = post.find_element_by_class_name('zncDM').text

            actions.move_to_element(post).perform()
            browser.implicitly_wait(15)
            time.sleep(code_sleep_time)
            eng_elements = post.find_elements_by_tag_name('li')

            for element in eng_elements:
                el_val, el_class = element.find_elements_by_tag_name('span')

                el_val = el_val.text
                el_class = el_class.get_attribute('class')

                if 'Play' in el_class:
                    approx_views = el_val
                elif 'Speech' in el_class:
                    approx_comments = el_val
                else:
                    approx_views = np.nan
                    approx_comments = np.nan

            vid_url = post.get_attribute('href')

            curr_row = pd.DataFrame({'title': title,
                                     'vid_length': vid_length,
                                     'approx_views': approx_views,
                                     'approx_comments': approx_comments,
                                     'vid_url': vid_url},
                                    index = [i])[['title','vid_length','approx_views','approx_comments','vid_url']]

            post_df = post_df.append(curr_row)
            actions.move_by_offset(-post.size['width']/2 - 1,-post.size['height']/2 - 1).perform()
        except:
            title = post.find_element_by_class_name('pu1E0').text
            vid_url = post.get_attribute('href')
            curr_failed_row = pd.DataFrame({'title': title,
                                            'vid_url': vid_url},
                                           index = [i])
            failed_posts = failed_posts.append(curr_failed_row)
        
        browser.implicitly_wait(15)
        time.sleep(code_sleep_time)
        run_time = round(time.time() - start_time, 2)

        update_progress((i + 1)/len(posts), '{0} Seconds {1} '.format(run_time, '{0}'.format(title)))
    
    return post_df, failed_posts

In [179]:
show = show_aliases.iloc[15,0]
show

'nbcsnl'

In [182]:
site = 'https://www.instagram.com/{show_alias}/channel/?hl=en'.format(show_alias = show)

browser = webdriver.Chrome(executable_path = chrome_dir + "\\chromedriver.exe", options=options)

browser.get(site)

browser.delete_all_cookies()

posts = browser.find_elements_by_class_name('_bz0w')

post_df, failed_posts = get_post_df(browser, posts)

browser.implicitly_wait(30)
time.sleep(10)

while True:
    curr_posts = browser.find_elements_by_class_name('_bz0w')
    indices_check = pd.DataFrame(list(filter(lambda x: x != None, [p.get_attribute('href') for p in curr_posts])), 
                                 columns = ['url']).merge(post_df[['vid_url']],
                                                          how = 'left',
                                                          left_on = ['url'],
                                                          right_on = 'vid_url')
    indices_check = indices_check[indices_check.vid_url.isnull()].index.tolist()

    if indices_check != []:
        
        curr_df, curr_failed_posts = get_post_df(browser, curr_posts)
        post_df = post_df.append(curr_df)
        failed_posts = failed_posts.append(curr_failed_posts)
        browser.implicitly_wait(15)
        time.sleep(code_sleep_time)
    else:
        break



Progress: [############################################################] 100.0% Elapsed Run Time: 37.79 Seconds Creating SNL: Hair and Makeup 


TypeError: 'WebElement' object is not iterable

In [186]:
curr_posts = browser.find_elements_by_class_name('_bz0w')
indices_check = pd.DataFrame(list(filter(lambda x: x != None, [p.get_attribute('href') for p in curr_posts])), 
                             columns = ['url']).merge(post_df[['vid_url']],
                                                      how = 'left',
                                                      left_on = ['url'],
                                                      right_on = 'vid_url')
indices_check = indices_check[indices_check.vid_url.isnull()].index.tolist()

In [195]:
list(enumerate([curr_posts]))

[(0,
  <selenium.webdriver.remote.webelement.WebElement (session="39a4ab1468c84a1df71416280f176473", element="d3e2d8e4-f1c8-4c84-b845-58f4eb2f470a")>)]

In [172]:
os.chdir('scraped_data')

post_df.reset_index(drop = True).to_csv('{show_alias}.csv'.format(show_alias = show))

os.chdir(chrome_dir)

browser.close()

In [178]:
show_aliases

,ig_handle
0,fallontonight
1,agt
2,nbcsongland
3,ninjawarrior
4,nbcbluffcitylaw
5,nbcbringthefunny
6,nbccouncilofdads
7,nbcindebted
8,nbclincoln
9,nbcperfectharmony
